# Genetic Algorithm Source Code and Demonstration

This notebook presents a problem generic implementation of the genetic algorithm. The Python classes are extended for the problem specific formulation of the N-Queens problem.

Table of Contents:
* [Chromosome Base Class](#Chromosome)
* [Genetic Algorithm Base Class](#Genetic-Algorithm)
* [N-Queens Formulation](#N-Queens-Formulation)
* [8-Queens Demonstration](#8-Queens-Demonstration)
* [16-Queens Demonstration](#16-Queens-Demonstration)

## Base Class Impelementations

### Chromosome

In [1]:
class Chromosome:
    """ 
    Implements an abstract chromosome class data type
    """
    
    def crossover(self, other):
        """
        Abstract method to implement cross-over between two chromosomes.
        @param other - other chromosome
        @return child1, child2 - output of the chromosome's children resultant from crossover
        """
        pass
    
    
    def mutate(self):
        """
        Abstract method to implement mutate operation on a chromosome.
        @return mutated chromosome
        """
        pass
    
    
    def getFitness(self):
        """
        Abstract method to implement the fitness evaluation for the chromosome.
        @return fitness value
        """
        pass
    
    def clone(self):
        """
        Abstract method to clone an instance of a chromosome
        """
        pass
    
    def __str__(self):
        """
        Abstract method to generate a string representation of the chromosome for printing.
        """
        pass

### Genetic Algorithm

In [9]:
import numpy as np
import sys

class GeneticAlgorithm:
    """
    Defines an abstract class for genetic algorithm.  All methods are implemetned
    except for buildInitialPopulation, which must be tailored to the Chromosome class
    used.
    """
    
    def __init__(self, popSize, generations, cross, mutate):
        """
        Initialization method for genetic algorithm class
        @param popSize - population size
        @param generations - number of generations the algorithm will run
        @param cross - probability of a crossover occuring during reproduction
        @param mutate - probability of a mutation occuring following reproduction
        """
        
        # Set GA parameter class variables
        self.populationSize = popSize
        self.numGenerations = generations
        self.probC = cross
        self.probM = cross
        
        # Initialize list class variables for population and roulette wheel
        self.population = [None for i in range(0,popSize)]
        self.roulette_min = [0 for i in range(0,popSize)]
        self.roulette_max = [0 for i in range(0,popSize)]
        
    def buildInitialPopulation(self):
        """
        Abstract method to generate a population of chromosomes.
        """
        pass
    
    def calculateRoulette(self):
        """
        Constructs a roulette wheel for parent selection.
        """
        
        # Determine the total fitness
        sum = 0
        for chromosome in self.population:
            sum = sum + chromosome.getFitness()
        
        # Generates roulette wheel where roulette_max[i] - roulette_min[i] == chromosome[i].getFitness()
        self.roulette_min[0] = 0
        for i in range(0, self.populationSize):
            if i != 0:
                self.roulette_min[i] = self.roulette_max[i-1]
            self.roulette_max[i] = self.roulette_min[i] + self.population[i].getFitness() / sum


    def pickChromosome(self):
        """
        Using roulette wheel, returns the index of a parent for reproduction.
        @return index of chromosome to reproduce.
        """
        spin = np.random.uniform()
        for i in range(0,self.populationSize):
            if spin > self.roulette_min[i] and spin <= self.roulette_max[i]:
                return i
        return self.populationSize-1
    

    def reproductionLoop(self):
        """ 
        Implements the GA algorithm's reproduction loop.  It is called once per generation.
        """
        newPop = []

        # Look through population populationSize/2 times
        #  each iteration generates two children
        for i in range(0, self.populationSize, 2):
            
            # Clone parents - Python copies by reference so we want to
            #  make sure we do not update the parents by mistake.
            x = self.population[self.pickChromosome()].clone()
            y = self.population[self.pickChromosome()].clone()
            
            # Crossover given crossover probabilty
            if (np.random.uniform() < self.probC):
                x, y = x.crossover(y)                
            
            # Mutate given mutate probability for each child
            if (np.random.uniform() < self.probM):
                x.mutate()
                
            if (np.random.uniform() < self.probM):
                y.mutate()
            
            # Add Children to new population
            newPop.append(x)
            newPop.append(y)
            
        # Update GA population with new population
        self.population = newPop    
    
    
    def getBest(self):
        """
        Prints the results of the current generation.  
        @return best chromosome
        """
        
        best = self.population[0]
        sum = 0
        fit = []
        
        for chromosome in self.population:
            sum = sum + chromosome.getFitness()
            if chromosome.getFitness() > best.getFitness():
                best = chromosome
                
        print("Best State: " + str(best.getFitness()) + ",  Avg. State: " + str(sum/self.populationSize))
        return best
    
    
    def runGA(self, target=sys.maxsize):
        """
        Implements the main GA population loop
        """
        
        # Initialize Variables toTrack best by generation and overall
        best = None
        bestOverall = None
        
        # Build initial poulation
        self.buildInitialPopulation()
        for i in range(0, self.numGenerations):

            # Generate roulette wheel for current population
            self.calculateRoulette()
        
            # Execute the GA reproduction loop for this generation
            self.reproductionLoop()
        
            # print generation's fitness and get best chromosome
            best = self.getBest()
            
            # Track the best
            if bestOverall is None:
                bestOverall = best
            elif best.getFitness() > bestOverall.getFitness():
                bestOverall = best
            
            # If target is reached, end algorithm
            if best.getFitness() >= target:
                print("Solution found at generation " + str(i))
                break
        
        # Prints the best overall solution
        print("Best overall Solution")
        print("Fitness: " + str(bestOverall.getFitness()))
        print(bestOverall)

## N-Queens Implementation and Demonstration

### N-Queens Formulation

This implementation does not use bits.  It specifically uses an array of size n specifying the column in which a queen is located for each row of the board.  

Crossover is implemented at the gene level (queen level) by slicing and concatenating between the two chromosomes at a random row.

Muation is implemented at the gene level by selecting a random row and changing its column to be a value from 0 to n-1, specifing a new column for the queen to reside.

In [10]:
import numpy as np

class QueenChromosome(Chromosome):
    
    def __init__(self, n, state=None):
        """
        Initialzies the queen chromosome.  If a child, it will clone the parent.  
        If a new chromosome, the values are initialized from 0 to n-1 for each gene.
        
        @param n - number of queens
        @param state - optional, state of parent (if child)
        """
        self.state = []
        self.numQueens = n
        
        # build new state
        if state == None:
            for i in range(0,n):
                self.state.append(np.random.randint(0,n))
        # Clone parent 
        else:
            self.state = state[:]
            
        # Call calculate fitness method and store in chromosome. (saves computational time)
        self.calculateFitness()
        
    def clone(self):
        """
        Creates a new queen by passing its number of queens and state to a new 
        nodes constructor
        @return cloned copy of node.
        """
        return QueenChromosome(self.numQueens, self.state)
    
    def crossover(self, other): 
        """
        Implements crossover for our queens chromosome formulation
        @return child1, child2 - two child nodes from self and other cross-over.
        """
        
        # Get Crossover point
        point = np.random.randint(0,self.numQueens)
        
        # split self
        x1 = self.state[0:point]
        x2 = self.state[point:]
        
        # split other
        y1 = other.state[0:point]
        y2 = other.state[point:]
        
        # crossover by appending sublists
        child1 = QueenChromosome(self.numQueens, x1 + y2)
        child2 = QueenChromosome(self.numQueens, y1 + x2)
        
        # return children
        return child1, child2
    
    def mutate(self):
        """
        Implements mutation on the chromosome.
        """
        # get point for mutation
        point = np.random.randint(0,self.numQueens)
                                  
        # Assign a new value at mutation point                          
        oldValue = self.state[point]
        while (oldValue == self.state[point]):
            self.state[point] = np.random.randint(0, self.numQueens)

            
    def calculateFitness(self):
        """
        Determines fitness of queens state.  For each queen i, it starts with a fitness of n-i
        and 1 point is deducted per conflicting queen j.  This value is summed for all queens.
        
        Max fitness is n x n-1
        
        @ return fitness.
        """
    
        # intialize
        self.fitness = 0
        rowFitness  = 0
        
        # Determine each row's fitness
        for row in range(0,self.numQueens):
            rowFitness = self.numQueens - 1 #initalize row fitness
                                  
            # check other rows and diagonals                   
            col = self.state[row]
            for i in range(0, self.numQueens):
                
                if i == row:
                    continue
                
                # Check for other rows with same column
                if (self.state[i] == col):
                    rowFitness = rowFitness - 1
                #diagonal 1
                if (self.state[i] == (col + (i-row))):
                    rowFitness = rowFitness - 1
                #diagonal 2
                if (self.state[i] == (col + (i-row))):
                    rowFitness = rowFitness - 1
        
            # add row fitness to self fitness.
            self.fitness = self.fitness + rowFitness

    

    def getFitness(self):
        """
        Returns the chromosome's stored fitness value.
        """
        return self.fitness

    
    def __str__(self):
        """
        Converts the state into a string representation of an N x N chess board
        where N is defined by the desired number of queens.
        
        Queen represented by a "Q".  Empty represented by a "."
        
        @return string representation of the node's chess board state
        """
        boardStr = ""
        
        # Traverse row by row
        for i in range(0,self.numQueens):
            # Initialize row as a string list with "." for blank
            row = ["." for x in range(0,self.numQueens)]
            
            # If row has queen, update list at queen index with Q
            if self.state[i] != -1:
                row[self.state[i]] = "Q"
                
            # Convert list to a string and concatenate
            boardStr += " ".join(map(str, row)) + "\n"
        return boardStr
    
    
    
class QueensGA(GeneticAlgorithm):
    """
    Implements the GA algorithm for the Queen chromosome.
    """
    
    def __init__(self, popSize, generations, cross, mutate, numQueens=8):
        """
        Initialization method for genetic algorithm class
        @param popSize - population size
        @param generations - number of generations the algorithm will run
        @param cross - probability of a crossover occuring during reproduction
        @param mutate - probability of a mutation occuring following reproduction
        @param numQueens - number of queens
        """
        self.numQueens = numQueens
        super().__init__(popSize, generations, cross, mutate)
    
    def buildInitialPopulation(self):
        """
        Builds the initial population of queen chromosomes.
        """
        self.population = []
        for i in range(self.populationSize):
            self.population.append(QueenChromosome(self.numQueens))

### 8-Queens Demonstration

Demonstrates the 8-Queens puzzle using a GA.  It may require multiple executions for the solution to converge on its target.

In [11]:
population = 50
generations = 1000
p_crossover = 0.7
p_mutation = 0.1
n_queens = 8

fit_target = 8*7

q = QueensGA(population, generations, p_crossover, p_mutation, n_queens)
q.runGA(target=fit_target)

Best State: 52,  Avg. State: 39.16
Best State: 54,  Avg. State: 40.16
Best State: 48,  Avg. State: 40.8
Best State: 52,  Avg. State: 40.72
Best State: 48,  Avg. State: 41.6
Best State: 54,  Avg. State: 41.88
Best State: 52,  Avg. State: 41.76
Best State: 52,  Avg. State: 42.04
Best State: 50,  Avg. State: 43.12
Best State: 52,  Avg. State: 44.16
Best State: 52,  Avg. State: 43.36
Best State: 54,  Avg. State: 42.04
Best State: 52,  Avg. State: 41.92
Best State: 50,  Avg. State: 43.04
Best State: 50,  Avg. State: 42.52
Best State: 52,  Avg. State: 41.64
Best State: 54,  Avg. State: 41.24
Best State: 52,  Avg. State: 41.52
Best State: 54,  Avg. State: 42.2
Best State: 50,  Avg. State: 43.04
Best State: 52,  Avg. State: 43.2
Best State: 52,  Avg. State: 43.04
Best State: 52,  Avg. State: 43.2
Best State: 52,  Avg. State: 45.0
Best State: 52,  Avg. State: 44.04
Best State: 52,  Avg. State: 42.72
Best State: 52,  Avg. State: 42.16
Best State: 50,  Avg. State: 41.24
Best State: 52,  Avg. Stat

### 16-Queens Demonstration

Demonstration of 16-Queens.  Convergence under this problem formulation is not likely.

In [12]:
population = 50
generations = 1000
p_crossover = 0.7
p_mutation = 0.1
n_queens = 16
fit_target = 16*15

q = QueensGA(population, generations, p_crossover, p_mutation, n_queens)
q.runGA(target=fit_target)

Best State: 220,  Avg. State: 206.0
Best State: 224,  Avg. State: 209.32
Best State: 222,  Avg. State: 206.76
Best State: 222,  Avg. State: 206.76
Best State: 230,  Avg. State: 205.96
Best State: 228,  Avg. State: 208.6
Best State: 222,  Avg. State: 208.36
Best State: 224,  Avg. State: 207.48
Best State: 224,  Avg. State: 207.04
Best State: 232,  Avg. State: 208.8
Best State: 224,  Avg. State: 209.8
Best State: 222,  Avg. State: 208.84
Best State: 222,  Avg. State: 209.84
Best State: 222,  Avg. State: 209.6
Best State: 220,  Avg. State: 208.36
Best State: 220,  Avg. State: 205.68
Best State: 224,  Avg. State: 205.8
Best State: 220,  Avg. State: 206.24
Best State: 224,  Avg. State: 205.36
Best State: 218,  Avg. State: 204.96
Best State: 222,  Avg. State: 205.48
Best State: 224,  Avg. State: 203.92
Best State: 218,  Avg. State: 202.64
Best State: 224,  Avg. State: 206.28
Best State: 226,  Avg. State: 208.08
Best State: 226,  Avg. State: 209.16
Best State: 226,  Avg. State: 211.88
Best St

Best State: 226,  Avg. State: 209.52
Best State: 224,  Avg. State: 209.52
Best State: 232,  Avg. State: 208.92
Best State: 222,  Avg. State: 206.24
Best State: 224,  Avg. State: 205.84
Best State: 222,  Avg. State: 203.96
Best State: 222,  Avg. State: 201.96
Best State: 218,  Avg. State: 199.92
Best State: 216,  Avg. State: 201.6
Best State: 216,  Avg. State: 201.44
Best State: 226,  Avg. State: 202.36
Best State: 224,  Avg. State: 202.2
Best State: 218,  Avg. State: 204.0
Best State: 218,  Avg. State: 203.48
Best State: 224,  Avg. State: 205.52
Best State: 228,  Avg. State: 205.68
Best State: 228,  Avg. State: 206.36
Best State: 228,  Avg. State: 204.96
Best State: 226,  Avg. State: 200.48
Best State: 216,  Avg. State: 200.4
Best State: 224,  Avg. State: 202.92
Best State: 218,  Avg. State: 201.6
Best State: 222,  Avg. State: 204.48
Best State: 222,  Avg. State: 204.92
Best State: 226,  Avg. State: 204.04
Best State: 218,  Avg. State: 202.48
Best State: 218,  Avg. State: 203.16
Best S

Best State: 226,  Avg. State: 207.16
Best State: 232,  Avg. State: 208.12
Best State: 230,  Avg. State: 209.72
Best State: 228,  Avg. State: 207.96
Best State: 224,  Avg. State: 207.12
Best State: 232,  Avg. State: 210.24
Best State: 224,  Avg. State: 208.24
Best State: 222,  Avg. State: 209.08
Best State: 230,  Avg. State: 212.2
Best State: 230,  Avg. State: 211.84
Best State: 222,  Avg. State: 209.48
Best State: 228,  Avg. State: 207.04
Best State: 226,  Avg. State: 206.04
Best State: 224,  Avg. State: 205.72
Best State: 228,  Avg. State: 204.72
Best State: 226,  Avg. State: 207.28
Best State: 232,  Avg. State: 207.72
Best State: 224,  Avg. State: 207.32
Best State: 224,  Avg. State: 205.8
Best State: 226,  Avg. State: 204.6
Best State: 226,  Avg. State: 206.72
Best State: 226,  Avg. State: 205.52
Best State: 226,  Avg. State: 207.52
Best State: 222,  Avg. State: 206.04
Best State: 224,  Avg. State: 208.6
Best State: 222,  Avg. State: 208.12
Best State: 224,  Avg. State: 208.12
Best 

Best State: 230,  Avg. State: 212.16
Best State: 230,  Avg. State: 211.84
Best State: 230,  Avg. State: 210.08
Best State: 224,  Avg. State: 210.4
Best State: 222,  Avg. State: 208.6
Best State: 222,  Avg. State: 207.92
Best State: 220,  Avg. State: 206.28
Best State: 224,  Avg. State: 206.84
Best State: 226,  Avg. State: 209.4
Best State: 226,  Avg. State: 209.92
Best State: 224,  Avg. State: 210.96
Best State: 224,  Avg. State: 210.52
Best State: 224,  Avg. State: 211.88
Best State: 224,  Avg. State: 210.4
Best State: 224,  Avg. State: 210.32
Best State: 224,  Avg. State: 208.52
Best State: 224,  Avg. State: 207.44
Best State: 226,  Avg. State: 208.12
Best State: 224,  Avg. State: 207.44
Best State: 224,  Avg. State: 207.72
Best State: 226,  Avg. State: 209.84
Best State: 222,  Avg. State: 210.84
Best State: 222,  Avg. State: 208.4
Best State: 226,  Avg. State: 208.36
Best State: 224,  Avg. State: 210.12
Best State: 222,  Avg. State: 210.28
Best State: 222,  Avg. State: 210.28
Best S

Best State: 222,  Avg. State: 206.68
Best State: 222,  Avg. State: 204.32
Best State: 220,  Avg. State: 203.84
Best State: 226,  Avg. State: 204.84
Best State: 222,  Avg. State: 206.0
Best State: 222,  Avg. State: 208.56
Best State: 220,  Avg. State: 209.04
Best State: 226,  Avg. State: 208.6
Best State: 222,  Avg. State: 208.36
Best State: 224,  Avg. State: 210.48
Best State: 226,  Avg. State: 210.48
Best State: 226,  Avg. State: 212.84
Best State: 226,  Avg. State: 210.6
Best State: 224,  Avg. State: 207.84
Best State: 226,  Avg. State: 206.96
Best State: 222,  Avg. State: 204.76
Best State: 222,  Avg. State: 206.12
Best State: 226,  Avg. State: 205.88
Best State: 228,  Avg. State: 204.92
Best State: 228,  Avg. State: 206.44
Best State: 224,  Avg. State: 207.44
Best State: 224,  Avg. State: 206.32
Best State: 222,  Avg. State: 205.56
Best State: 220,  Avg. State: 205.04
Best State: 222,  Avg. State: 206.32
Best State: 228,  Avg. State: 207.68
Best State: 228,  Avg. State: 208.56
Best

## Multivariate Optimization Demonstration

This section of the notebook implements case study 1 from the lecture notes.

In [13]:
import numpy as np
import math

class DemoChromosome(Chromosome):
    """
    Demonstration of a multi-variate optimization problem as shown in case study #2
    """
    
    def __init__(self, chromosome=None):
        """
        Initializes a new chromosome. 
        @param chromosome - string representation of chromosome to be used in new chromosome
        """
        if chromosome:
            self.chromosome = chromosome[:]
        else:
            self.chromosome = self.getRandomChromosomeString()
            
    def getRandomChromosomeString(self):
        """
        Method that returns a random chromsomesome given the problem formulation
        @return new randomly generated chromosome
        """
        x = -3 + 6* np.random.uniform()
        y = -3 + 6* np.random.uniform()
        return self.encode(x,y)
    
    
    def encode(self, x, y):
        """
        Encodes two floating point numbers as a single chromosome string of length 16
        @param x - floating point number
        @param y - floating point number
        @return encoded string
        """
        return self.encodeGene(x) + self.encodeGene(y)
    
    
    def decode(self, valstr):
        """
        Decondes chromosome string into two floating point numbers.
        @return xval, yval - two string values represented in chromosome.
        """
        xval = self.decodeGene(valstr[:8])
        yval = self.decodeGene(valstr[8:])
        return xval, yval
    
    
    def encodeGene(self, valf):
        """
        Encodes a floating point value from -3 to 3 to an 8 bit binary string
        @param valf - floating point value
        @return 8-bit string representation of float.
        """
        
        # Convert value from -3 to 3 to a 8 bit number
        valint = int((valf + 3)/6 * 255)
        valstr = format(valint,'b')
        while (len(valstr) < 8):
            valstr = '0'+valstr
        return valstr
    
    
    def decodeGene(self, valstr):
        """
        Decods an 8-bit string into its floating point representation between -3 to 3
        """
        valint = int(valstr,2)
        valf = valint*6/255 - 3
        return valf
        
    def crossover(self, other):
        """
        Method to implement cross-over between two chromosomes.
        @param other - other chromosome
        @return child1, child2 - output of the chromosome's children resultant from crossover
        """
        point = np.random.randint(0,len(self.chromosome))
        new1 = self.chromosome[:point] + other.chromosome[point:]
        new2 = other.chromosome[:point] + self.chromosome[point:]
        return DemoChromosome(new1), DemoChromosome(new2)
    
    
    def mutate(self):
        """
        Method to implement mutate operation on a chromosome.
        @return mutated chromosome
        """
        point = np.random.randint(0,len(self.chromosome))
        if self.chromosome[point] == '0':
            bit = '1'
        else:
            bit = '0'
            
        self.chromosome = self.chromosome[:point] + bit + self.chromosome[(point+1):]
    
    def getFitness(self):
        """
        Method to implement the fitness evaluation for the chromosome.
        @return fitness value
        """
        x, y = self.decode(self.chromosome)
        return ((1-x)**2)*math.exp(-1*(x**2)-(y+1)**2 - (x - x**3 - y**3)*math.exp(-1*x**2 - y**2))
    
    def clone(self):
        """
        Method to clone an instance of a chromosome
        """
        return DemoChromosome(self.chromosome[:])
    
    def __str__(self):
        """
        Abstract method to generate a string representation of the chromosome for printing.
        """
        x, y = self.decode(self.chromosome)
        return self.chromosome + ", (" + str(x) + ", " + str(y)+ "), " + str(self.getFitness())
    
    
class DemoGA(GeneticAlgorithm):
    """
    Implements the GA algorithm for the Queen chromosome.
    """
    
   
    def buildInitialPopulation(self):
        """
        Builds the initial population of queen chromosomes.
        """
        self.population = []
        for i in range(self.populationSize):
            self.population.append(DemoChromosome())

In [17]:
population = 50
generations = 1000
p_crossover = 0.7
p_mutation = 0.1

d = DemoGA(population, generations, p_crossover, p_mutation)
d.runGA()

Best State: 1.6341155463311599,  Avg. State: 0.6391640794045031
Best State: 1.6341155463311599,  Avg. State: 0.911030225297173
Best State: 1.6368732486715813,  Avg. State: 1.0368290580048285
Best State: 1.6544358177835203,  Avg. State: 0.942178852438821
Best State: 1.6481622621366232,  Avg. State: 1.0639628117436668
Best State: 1.6042340326592475,  Avg. State: 0.9903799740580815
Best State: 1.6300334019654874,  Avg. State: 0.9785113407345308
Best State: 1.662267287639255,  Avg. State: 1.0651902767983212
Best State: 1.6608795956088949,  Avg. State: 1.0259998690074048
Best State: 1.6429353889332072,  Avg. State: 1.081539520725154
Best State: 1.6593039305589985,  Avg. State: 1.0308739185968356
Best State: 1.6593039305589985,  Avg. State: 1.1196519504743374
Best State: 1.5853190388175025,  Avg. State: 1.0033730711776676
Best State: 1.5810518407547662,  Avg. State: 1.0064035957980115
Best State: 1.6376615409286632,  Avg. State: 0.961292080919647
Best State: 1.648877270250742,  Avg. State: 1

Best State: 1.6433276904300993,  Avg. State: 1.007341689011744
Best State: 1.6433276904300993,  Avg. State: 1.019179311928674
Best State: 1.6433276904300993,  Avg. State: 1.0197353199509045
Best State: 1.6433276904300993,  Avg. State: 0.9011496762401684
Best State: 1.6415378445900402,  Avg. State: 1.0699746806860706
Best State: 1.6517758698674907,  Avg. State: 1.0483438765186515
Best State: 1.648877270250742,  Avg. State: 0.9676661981110309
Best State: 1.6433276904300993,  Avg. State: 0.9746918085031929
Best State: 1.6565460696498808,  Avg. State: 1.0764378740786655
Best State: 1.6540565053814424,  Avg. State: 1.0188370625692742
Best State: 1.6540565053814424,  Avg. State: 0.9155200804454023
Best State: 1.6569742626187534,  Avg. State: 1.0284193974684244
Best State: 1.6300460366034364,  Avg. State: 1.0503222265566563
Best State: 1.6600637153260056,  Avg. State: 1.114596765197642
Best State: 1.6607676646922942,  Avg. State: 1.0657162513606095
Best State: 1.6569742626187534,  Avg. State:

Best State: 1.6614238191568147,  Avg. State: 1.1231382521544773
Best State: 1.6614238191568147,  Avg. State: 0.9163909020423575
Best State: 1.6614238191568147,  Avg. State: 1.0093510804982606
Best State: 1.6614238191568147,  Avg. State: 0.9655657727718537
Best State: 1.6614238191568147,  Avg. State: 1.0925907713401408
Best State: 1.6608795956088949,  Avg. State: 1.1930323841962034
Best State: 1.6614238191568147,  Avg. State: 1.0620016385139672
Best State: 1.6616426252108765,  Avg. State: 0.9577264014824638
Best State: 1.6616426252108765,  Avg. State: 0.9805518572045545
Best State: 1.6616426252108765,  Avg. State: 1.0876682563621332
Best State: 1.6616426252108765,  Avg. State: 1.1388251563004916
Best State: 1.6575714418895355,  Avg. State: 1.0570195374878282
Best State: 1.6433276904300993,  Avg. State: 0.9671943238568036
Best State: 1.6591186254121535,  Avg. State: 1.084657010578288
Best State: 1.6491808351324821,  Avg. State: 0.9551248650888162
Best State: 1.6608795956088949,  Avg. Sta

Best State: 1.662267287639255,  Avg. State: 1.0802742936886573
Best State: 1.662267287639255,  Avg. State: 1.1357203034267467
Best State: 1.662267287639255,  Avg. State: 1.1937781919490418
Best State: 1.6608795956088949,  Avg. State: 1.2064653107861616
Best State: 1.662267287639255,  Avg. State: 1.1304641138304121
Best State: 1.6608795956088949,  Avg. State: 1.0427715324208884
Best State: 1.662267287639255,  Avg. State: 1.071485670353256
Best State: 1.6565266905318579,  Avg. State: 1.0088595979315178
Best State: 1.6573753297798737,  Avg. State: 1.0669367855024006
Best State: 1.6569742626187534,  Avg. State: 1.1371164116748425
Best State: 1.6565266905318579,  Avg. State: 1.0357140819862252
Best State: 1.652707479807598,  Avg. State: 0.8711621794225668
Best State: 1.6575714418895355,  Avg. State: 1.1207294358078028
Best State: 1.6591186254121535,  Avg. State: 1.0292849680752159
Best State: 1.65914045878938,  Avg. State: 1.2145714579502376
Best State: 1.662267287639255,  Avg. State: 1.033

Best State: 1.6517758698674907,  Avg. State: 1.004419162727749
Best State: 1.6569742626187534,  Avg. State: 0.9507760962475926
Best State: 1.6569742626187534,  Avg. State: 1.1474603855120618
Best State: 1.649590661367342,  Avg. State: 1.2236842982186125
Best State: 1.6548148602615131,  Avg. State: 1.0581249871549945
Best State: 1.6540565053814424,  Avg. State: 1.16019164589062
Best State: 1.6608795956088949,  Avg. State: 1.0875113630502495
Best State: 1.6608795956088949,  Avg. State: 1.2101611393856175
Best State: 1.6540565053814424,  Avg. State: 1.0347546731551163
Best State: 1.6569742626187534,  Avg. State: 1.0716981652669335
Best State: 1.6429353889332072,  Avg. State: 0.9793965777242519
Best State: 1.6429353889332072,  Avg. State: 0.9327928631766753
Best State: 1.649590661367342,  Avg. State: 1.10294926114507
Best State: 1.6565460696498808,  Avg. State: 1.0843778848610377
Best State: 1.6565460696498808,  Avg. State: 1.0083245968656116
Best State: 1.6448830690981102,  Avg. State: 1.